In [18]:
import warnings ## funciones para imprimir y registrar advertencias.
warnings.filterwarnings("ignore") #suprimir cualquier advertencia que pueda ser generada por el código.

In [19]:
###Librerias
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import a_funciones as funciones  ###archivo de funciones propias
import sys ## saber ruta de la que carga paquetes

%matplotlib inline

**Preprocesamiento de datos**



In [20]:
###Lectura de datos 
base = pd.read_csv("https://raw.githubusercontent.com/andressj1/A-RH/main/Bases/base2015.csv", sep=",")
base1 = pd.read_csv("https://raw.githubusercontent.com/andressj1/A-RH/main/Bases/base2016.csv", sep=",")


## **Preprocesamiento para la base final**

In [21]:
base.head(3)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,año_x,Age,BusinessTravel,Department,DistanceFromHome,Education,...,YearsWithCurrManager,año_y,JobInvolvement,PerformanceRating,año_x.1,Unnamed: 0,Attrition,retirementType,resignationReason,año_y.1
0,1,3.0,4.0,2.0,2015,51,Travel_Rarely,Sales,6,2,...,0,2015,3,3,2015,NaN,NaN,NaN,NaN,NaN
1,2,3.0,2.0,4.0,2015,31,Travel_Frequently,Research & Development,10,1,...,4,2015,2,4,2015,0.0,Yes,Resignation,Others,2016.0
2,3,2.0,2.0,1.0,2015,32,Travel_Frequently,Research & Development,17,4,...,3,2015,3,3,2015,NaN,NaN,NaN,NaN,NaN


In [22]:
# Obtener la cantidad de valores únicos en cada columna; solo se aplico a esta base debido a la cantidad de variables
nunique_values = base.nunique()

# Seleccionar las columnas con un solo valor
single_value_columns = nunique_values[nunique_values == 1].index

print("Variables con un único valor: ", single_value_columns)

Variables con un único valor:  Index(['año_x', 'año_y', 'año_x.1', 'Attrition', 'año_y.1'], dtype='object')


In [23]:
###Eliminacion de variables que no aportaria en el modelo
columns_to_drop = ['año_x', 'año_y', 'año_x.1', 'Unnamed: 0','año_y.1','EmployeeID']
base = base.drop(columns=columns_to_drop)

In [24]:
## Verificación de valores nulos
print(base.isnull().sum())

EnvironmentSatisfaction       0
JobSatisfaction               0
WorkLifeBalance               0
Age                           0
BusinessTravel                0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
Gender                        0
JobLevel                      0
JobRole                       0
MaritalStatus                 0
MonthlyIncome                 0
NumCompaniesWorked            0
PercentSalaryHike             0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
YearsAtCompany                0
YearsSinceLastPromotion       0
YearsWithCurrManager          0
JobInvolvement                0
PerformanceRating             0
Attrition                  3760
retirementType             3760
resignationReason          3760
dtype: int64


In [25]:
## Verificación de valores nulos
print(base.isnull().sum())


EnvironmentSatisfaction       0
JobSatisfaction               0
WorkLifeBalance               0
Age                           0
BusinessTravel                0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
Gender                        0
JobLevel                      0
JobRole                       0
MaritalStatus                 0
MonthlyIncome                 0
NumCompaniesWorked            0
PercentSalaryHike             0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
YearsAtCompany                0
YearsSinceLastPromotion       0
YearsWithCurrManager          0
JobInvolvement                0
PerformanceRating             0
Attrition                  3760
retirementType             3760
resignationReason          3760
dtype: int64


In [26]:
base['retirementType'].value_counts()

Resignation    585
Fired           65
Name: retirementType, dtype: int64

In [27]:
#### se procede a eliminar la categoria "fired" porque no aporta nada un empleado que despedimos
base = base[base['retirementType'] != 'Fired']


In [28]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4345 entries, 0 to 4409
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EnvironmentSatisfaction  4345 non-null   float64
 1   JobSatisfaction          4345 non-null   float64
 2   WorkLifeBalance          4345 non-null   float64
 3   Age                      4345 non-null   int64  
 4   BusinessTravel           4345 non-null   object 
 5   Department               4345 non-null   object 
 6   DistanceFromHome         4345 non-null   int64  
 7   Education                4345 non-null   int64  
 8   EducationField           4345 non-null   object 
 9   Gender                   4345 non-null   object 
 10  JobLevel                 4345 non-null   int64  
 11  JobRole                  4345 non-null   object 
 12  MaritalStatus            4345 non-null   object 
 13  MonthlyIncome            4345 non-null   int64  
 14  NumCompaniesWorked      

## **Preprocesamiento para la base2**

In [29]:
base1.head(3)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,año_x,Age,BusinessTravel,Department,DistanceFromHome,Education,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,año_y,JobInvolvement,PerformanceRating,año
0,1,3.0,4.0,2.0,2015,51,Travel_Rarely,Sales,6,2,...,0,1.0,6,1,0,0,2016,3,3,2016
1,2,3.0,2.0,4.0,2015,31,Travel_Frequently,Research & Development,10,1,...,1,6.0,3,5,1,4,2016,2,4,2016
2,3,2.0,2.0,1.0,2015,32,Travel_Frequently,Research & Development,17,4,...,3,5.0,2,5,0,3,2016,3,3,2016


In [30]:
# Obtener la cantidad de valores únicos en cada columna; solo se aplico a esta base debido a la cantidad de variables
nunique_values = base1.nunique()

# Seleccionar las columnas con un solo valor
single_value_columns = nunique_values[nunique_values == 1].index

print("Variables con un único valor: ", single_value_columns)

Variables con un único valor:  Index(['año_x', 'año_y', 'año'], dtype='object')


In [31]:
###Eliminacion de variables que no aportaria en el modelo
columns_to_drop = ['año_x', 'año_y', 'año','EmployeeID']
base1 = base1.drop(columns=columns_to_drop)

In [32]:
## Verificación de valores nulos
print(base1.isnull().sum())

EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
Age                        0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
JobInvolvement             0
PerformanceRating          0
dtype: int64


In [33]:
####Exportacion de bases 
base.to_csv(r'C:\Users\luisa\OneDrive\Desktop\Analitica lll RH\Bases\basefinal.csv', index=False)
base1.to_csv(r'C:\Users\luisa\OneDrive\Desktop\Analitica lll RH\Bases\base16.csv', index=False)